The HW2VEC codebase is available at https://github.com/AICPS/hw2vec

**Step 1:** To set up the environment for HW2VEC, the following bash commands were executed in the following sequence in the terminal:

`conda create --name hw2vec_python37_pyg_pygraphviz`

`conda activate hw2vec_python37_pyg_pygraphviz`

`conda install python=3.7`

`conda install pyg -c pyg`

`python3 -m pip install pygraphviz`

`python3 -m pip install pydot pathlib patool pyunpack pyverilog`

`python3 -m pip install pytorch-nlp`

`python3 -m pip install scikit-image`

`python3 -m pip install pandas networkx matplotlib`

`conda list`

*Note:* 
1. The package versions and the commands mentioned in the HW2VEC repo cause lot of errors as they are quite old. 

2. My system has CUDA 12.

**Step 2:** Modify line 105 of hw2vec/graph2vec/models.py as:

`self.load_state_dict(torch.load(model_weight_path, map_location=torch.device('cpu')),strict=False)`

**Step 3:** Modify line 28 of hw2vec/examples/use_case_1.py as:

`hw_design_dir_path = Path("../assets/datasets/TJ-RTL-toy/TjFree/RS232/")`

`RS232` can be replaced with other example circuits present in the given path.

**Step 4:** Unzip the assets/datasets.zip

**Step 5:** Execute the following command in the terminal:

`python use_case_1.py`